In [1]:
import pandas as pd

In [2]:
gameId=21900895

In [3]:
games_df=pd.read_csv('./data/games.csv')  #主場和客場的總體數據
games_details_df=pd.read_csv('./data/games_details.csv')# 有上場球員的比賽詳細數據
players_df=pd.read_csv('./data/players.csv')# PLAYER_NAME|TEAM_ID|PLAYER_ID|SEASON
# TEAM_ID| LEAGUE_ID| SEASON_ID| STANDINGSDATE| CONFERENCE| TEAM| G| W| L| W_PCT| HOME_RECORD| ROAD_RECORD
ranking_df=pd.read_csv('./data/ranking.csv')
teams_df=pd.read_csv('./data/teams.csv') 
teams_name_df=pd.read_csv('./data/team_name_details.csv').drop(['Unnamed: 0'],axis=1)

In [4]:
ranking_df.loc[ranking_df['SEASON_ID']==22019]
# G:W+L       # 總場數
# W:Wins      # 贏的場數
# L:Losses    # 輸的場數
# PCT:Winning Percentage(w/G)
# GB:Games Back  #該球隊落後賽區冠軍的場次，是由該賽區冠軍的淨勝場次減去該隊的淨勝場次之差除以2。
# HOME:Home Record(HW-HL)主場
# AWAY:Away Record(AW-AL)客場
# =========================================
# DIV:Division Record    
# 在NBA東區(Eastern Conference) Atlantic=大西洋組 Central=中央組 Southeast=東南組
# 在NBA西區(Western Conference)Northwest=西北組 Pacific=太平洋組 Southwest=西南組
# CONF:Conference Record (東西區)
# PPG:Points Per Game
# OPP PPG:Opponent Points Per Game
# DIFF:Average Point Differential
# STRK:Current Streak  # 為連勝或連敗之場數，W 為勝；L 為敗
# L10:Record last 10 games  # 近10場之勝敗紀錄

# Z --:Clinched Conference

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD
0,1610612747,0,22019,2020-03-01,West,L.A. Lakers,59,46,13,0.780,21-7,25-6
1,1610612743,0,22019,2020-03-01,West,Denver,60,41,19,0.683,24-7,17-12
2,1610612746,0,22019,2020-03-01,West,LA Clippers,60,41,19,0.683,25-6,16-13
3,1610612745,0,22019,2020-03-01,West,Houston,59,39,20,0.661,21-8,18-12
4,1610612762,0,22019,2020-03-01,West,Utah,59,37,22,0.627,21-9,16-13
...,...,...,...,...,...,...,...,...,...,...,...,...
151373,1610612749,0,22019,2019-10-22,East,Milwaukee,0,0,0,0.000,0-0,0-0
151374,1610612752,0,22019,2019-10-22,East,New York,0,0,0,0.000,0-0,0-0
151375,1610612753,0,22019,2019-10-22,East,Orlando,0,0,0,0.000,0-0,0-0
151376,1610612755,0,22019,2019-10-22,East,Philadelphia,0,0,0,0.000,0-0,0-0


In [5]:
from nba_api.stats.library.parameters import LeagueID, Season, SeasonType, SeasonNullable
SeasonType??

In [6]:
from nba_api.stats.endpoints import leaguestandings
items = leaguestandings.LeagueStandings(league_id="00",season="2019-20",season_type="Regular Season")
items_df = items.get_data_frames()[0]
items_df.head()
# items_df.columns

,LeagueID,SeasonID,TeamID,TeamCity,TeamName,Conference,ConferenceRecord,PlayoffRank,ClinchIndicator,Division,...,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,PreAS,PostAS
0,00,22019,1610612747,Los Angeles,Lakers,West,36-10,1,- w,Pacific,...,None,None,1-0,2-5,None,3-1,14-1,9-5,41-12,11-7
1,00,22019,1610612749,Milwaukee,Bucks,East,37-7,1,- e,Central,...,None,None,1-0,2-5,None,2-2,15-1,13-2,46-8,10-9
2,00,22019,1610612746,LA,Clippers,West,32-16,2,- x,Pacific,...,None,None,0-1,5-2,None,4-2,10-4,10-5,37-18,12-5
3,00,22019,1610612761,Toronto,Raptors,East,34-11,2,- a,Atlantic,...,None,None,None,7-1,None,4-1,10-3,9-7,40-15,13-4
4,00,22019,1610612743,Denver,Nuggets,West,29-16,3,- nw,Northwest,...,None,None,None,3-5,None,3-2,10-2,10-6,38-17,8-10


In [7]:
# 用中文查詢 team 資料
def findNickNameFromChinese(word=''):
    tg_df = teams_name_df.loc[teams_name_df['chinese name'].str.contains(word)]
    return tg_df.T.to_dict()[tg_df.index[0]]
    

In [8]:
# 抓球隊中文名
def findChineneName(nick):
    return teams_name_df.loc[teams_name_df['nickname']==nick, 'chinese name'].values[0]

In [9]:
# 抓比賽主客隊簡名
def findHomeAwayTeamName(gameid):
    one_df = games_df.loc[games_df['GAME_ID']==21900895,:]
    datadict = {
        'HOME_TEAM_ID':one_df['HOME_TEAM_ID'].values[0],
        'VISITOR_TEAM_ID':one_df['VISITOR_TEAM_ID'].values[0],
        'HOME_TEAM_Name':games_details_df.loc[(games_details_df['GAME_ID']==gameid) & 
                                              (games_details_df['TEAM_ID']==one_df['HOME_TEAM_ID'].values[0]),
                                              'TEAM_ABBREVIATION'].unique()[0],
        'VISITOR_TEAM_Name':games_details_df.loc[(games_details_df['GAME_ID']==gameid) & 
                                              (games_details_df['TEAM_ID']==one_df['VISITOR_TEAM_ID'].values[0]),
                                              'TEAM_ABBREVIATION'].unique()[0],
        
    }
    return datadict

In [10]:
# 抓特定比賽的球員名單
def getPlayerListOnGame(gameid,teamid):
    playerongame_df = games_details_df.loc[games_details_df['GAME_ID']==gameid]
    return playerongame_df.loc[playerongame_df['TEAM_ID']== teamid,'PLAYER_ID'].to_list()

In [11]:
# 抓特定比賽的球員數據
def getGame2PlayerDetails(gameid,playerlist):
    return games_details_df.loc[(games_details_df['GAME_ID']==gameid) & (games_details_df['PLAYER_ID'].isin(playerlist))]

In [12]:
games_df

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2020-03-01,21900895,Final,1610612766,1610612749,2019,1610612766,85.0,0.354,0.900,...,22.0,47.0,1610612749,93.0,0.402,0.762,0.226,20.0,61.0,0
1,2020-03-01,21900896,Final,1610612750,1610612742,2019,1610612750,91.0,0.364,0.400,...,19.0,57.0,1610612742,111.0,0.468,0.632,0.275,28.0,56.0,0
2,2020-03-01,21900897,Final,1610612746,1610612755,2019,1610612746,136.0,0.592,0.805,...,25.0,37.0,1610612755,130.0,0.505,0.650,0.488,27.0,37.0,1
3,2020-03-01,21900898,Final,1610612743,1610612761,2019,1610612743,133.0,0.566,0.700,...,38.0,41.0,1610612761,118.0,0.461,0.897,0.263,24.0,36.0,1
4,2020-03-01,21900899,Final,1610612758,1610612765,2019,1610612758,106.0,0.407,0.885,...,18.0,51.0,1610612765,100.0,0.413,0.667,0.429,23.0,42.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23190,2014-10-06,11400007,Final,1610612737,1610612740,2014,1610612737,93.0,0.419,0.821,...,24.0,50.0,1610612740,87.0,0.366,0.643,0.375,17.0,43.0,1
23191,2014-10-06,11400004,Final,1610612741,1610612764,2014,1610612741,81.0,0.338,0.719,...,18.0,40.0,1610612764,85.0,0.411,0.636,0.267,17.0,47.0,0
23192,2014-10-06,11400005,Final,1610612747,1610612743,2014,1610612747,98.0,0.448,0.682,...,29.0,45.0,1610612743,95.0,0.387,0.659,0.500,19.0,43.0,1
23193,2014-10-05,11400002,Final,1610612761,1610612758,2014,1610612761,99.0,0.440,0.771,...,21.0,30.0,1610612758,94.0,0.469,0.725,0.385,18.0,45.0,1


In [13]:
games_details_df

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,21900895,1610612749,MIL,Milwaukee,202083,Wesley Matthews,F,NaN,27:08,3.0,...,4.0,4.0,8.0,2.0,2.0,0.0,0.0,0.0,8.0,11.0
1,21900895,1610612749,MIL,Milwaukee,203507,Giannis Antetokounmpo,F,NaN,34:55,17.0,...,2.0,18.0,20.0,6.0,1.0,0.0,3.0,2.0,41.0,22.0
2,21900895,1610612749,MIL,Milwaukee,201572,Brook Lopez,C,NaN,26:25,4.0,...,2.0,5.0,7.0,0.0,0.0,3.0,0.0,2.0,16.0,16.0
3,21900895,1610612749,MIL,Milwaukee,1628978,Donte DiVincenzo,G,NaN,27:35,1.0,...,1.0,6.0,7.0,5.0,0.0,1.0,2.0,0.0,2.0,14.0
4,21900895,1610612749,MIL,Milwaukee,202339,Eric Bledsoe,G,NaN,22:17,2.0,...,1.0,0.0,1.0,2.0,1.0,0.0,3.0,2.0,4.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576777,11200005,1610612743,DEN,Denver,202706,Jordan Hamilton,NaN,NaN,19,4.0,...,0.0,2.0,2.0,0.0,2.0,0.0,1.0,3.0,17.0,NaN
576778,11200005,1610612743,DEN,Denver,202702,Kenneth Faried,NaN,NaN,23,7.0,...,1.0,0.0,1.0,1.0,1.0,0.0,3.0,3.0,18.0,NaN
576779,11200005,1610612743,DEN,Denver,201585,Kosta Koufos,NaN,NaN,15,3.0,...,3.0,5.0,8.0,0.0,1.0,0.0,0.0,3.0,6.0,NaN
576780,11200005,1610612743,DEN,Denver,202389,Timofey Mozgov,NaN,NaN,19,1.0,...,1.0,2.0,3.0,1.0,0.0,0.0,4.0,2.0,2.0,NaN


In [14]:
players_df

,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON
0,Royce O'Neale,1610612762,1626220,2019
1,Bojan Bogdanovic,1610612762,202711,2019
2,Rudy Gobert,1610612762,203497,2019
3,Donovan Mitchell,1610612762,1628378,2019
4,Mike Conley,1610612762,201144,2019
...,...,...,...,...
7223,Lanny Smith,1610612758,201831,2009
7224,Warren Carter,1610612752,201999,2009
7225,Bennet Davis,1610612751,201834,2009
7226,Brian Hamilton,1610612751,201646,2009


In [15]:
ranking_df

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD
0,1610612747,0,22019,2020-03-01,West,L.A. Lakers,59,46,13,0.780,21-7,25-6
1,1610612743,0,22019,2020-03-01,West,Denver,60,41,19,0.683,24-7,17-12
2,1610612746,0,22019,2020-03-01,West,LA Clippers,60,41,19,0.683,25-6,16-13
3,1610612745,0,22019,2020-03-01,West,Houston,59,39,20,0.661,21-8,18-12
4,1610612762,0,22019,2020-03-01,West,Utah,59,37,22,0.627,21-9,16-13
...,...,...,...,...,...,...,...,...,...,...,...,...
179528,1610612765,0,22013,2014-09-01,East,Detroit,82,29,53,0.354,17-24,12-29
179529,1610612738,0,22013,2014-09-01,East,Boston,82,25,57,0.305,16-25,9-32
179530,1610612753,0,22013,2014-09-01,East,Orlando,82,23,59,0.280,19-22,4-37
179531,1610612755,0,22013,2014-09-01,East,Philadelphia,82,19,63,0.232,10-31,9-32


In [16]:
teams_df

,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,1610612738,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,0,1610612740,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
3,0,1610612741,1966,2019,CHI,Bulls,1966,Chicago,United Center,21711.0,Jerry Reinsdorf,Gar Forman,Jim Boylen,Windy City Bulls
4,0,1610612742,1980,2019,DAL,Mavericks,1980,Dallas,American Airlines Center,19200.0,Mark Cuban,Donnie Nelson,Rick Carlisle,Texas Legends
5,0,1610612743,1976,2019,DEN,Nuggets,1976,Denver,Pepsi Center,19099.0,Stan Kroenke,Tim Connelly,Michael Malone,No Affiliate
6,0,1610612745,1967,2019,HOU,Rockets,1967,Houston,Toyota Center,18104.0,Tilman Fertitta,Daryl Morey,Mike D'Antoni,Rio Grande Valley Vipers
7,0,1610612746,1970,2019,LAC,Clippers,1970,Los Angeles,Staples Center,19060.0,Steve Ballmer,Michael Winger,Doc Rivers,Agua Caliente Clippers of Ontario
8,0,1610612747,1948,2019,LAL,Lakers,1948,Los Angeles,Staples Center,19060.0,Jerry Buss Family Trust,Rob Pelinka,Frank Vogel,South Bay Lakers
9,0,1610612748,1988,2019,MIA,Heat,1988,Miami,AmericanAirlines Arena,19600.0,Micky Arison,Pat Riley,Erik Spoelstra,Sioux Falls Skyforce


In [17]:
# test_df['TEAM_ABBREVIATION'].unique()
dd=games_df.loc[games_df['GAME_ID']==21900895,:]
tt = findHomeAwayTeamName(21900895)
tt

{'HOME_TEAM_ID': 1610612766,
 'VISITOR_TEAM_ID': 1610612749,
 'HOME_TEAM_Name': 'CHA',
 'VISITOR_TEAM_Name': 'MIL'}

In [18]:
test_df = games_details_df.loc[games_details_df['GAME_ID']==21900895]
test_df
playerList = getPlayerListOnGame(gameid=gameId,teamid=tt['HOME_TEAM_ID'])
playerList

[1628970,
 1629023,
 202687,
 1628984,
 1626179,
 1628998,
 1629667,
 1626195,
 1628997,
 201587,
 1629147,
 1628370,
 203469]

In [19]:
test_df.loc[test_df['TEAM_ID']== 1610612749,'PLAYER_ID']

0      202083
1      203507
2      201572
3     1628978
4      202339
5     1626192
6      201577
7     1628425
8      101107
9      201588
10     203648
11     101141
12    1628391
Name: PLAYER_ID, dtype: int64

In [20]:
# 抓特定比賽的球員數據
targetPlayer_df = getGame2PlayerDetails(gameid=gameId,playerlist=playerList)
targetPlayer_df

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
13,21900895,1610612766,CHA,Charlotte,1628970,Miles Bridges,F,NaN,35:15,3.0,...,1.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,7.0,-4.0
14,21900895,1610612766,CHA,Charlotte,1629023,P.J. Washington,F,NaN,31:52,5.0,...,1.0,5.0,6.0,3.0,0.0,2.0,2.0,3.0,12.0,-13.0
15,21900895,1610612766,CHA,Charlotte,202687,Bismack Biyombo,C,NaN,22:07,2.0,...,4.0,5.0,9.0,2.0,0.0,2.0,1.0,3.0,8.0,-15.0
16,21900895,1610612766,CHA,Charlotte,1628984,Devonte' Graham,G,NaN,32:21,7.0,...,0.0,2.0,2.0,3.0,1.0,0.0,0.0,2.0,17.0,-14.0
17,21900895,1610612766,CHA,Charlotte,1626179,Terry Rozier,G,NaN,36:05,6.0,...,2.0,1.0,3.0,4.0,1.0,0.0,2.0,2.0,13.0,-20.0
18,21900895,1610612766,CHA,Charlotte,1628998,Cody Martin,NaN,NaN,29:08,4.0,...,0.0,5.0,5.0,2.0,0.0,1.0,1.0,2.0,11.0,2.0
19,21900895,1610612766,CHA,Charlotte,1629667,Jalen McDaniels,NaN,NaN,9:21,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,2.0,0.0
20,21900895,1610612766,CHA,Charlotte,1626195,Willy Hernangomez,NaN,NaN,20:21,4.0,...,3.0,10.0,13.0,4.0,2.0,0.0,1.0,0.0,10.0,11.0
21,21900895,1610612766,CHA,Charlotte,1628997,Caleb Martin,NaN,NaN,23:30,2.0,...,1.0,3.0,4.0,1.0,1.0,0.0,1.0,3.0,5.0,13.0
22,21900895,1610612766,CHA,Charlotte,201587,Nicolas Batum,NaN,DNP - Coach's Decision,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
test_df

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,21900895,1610612749,MIL,Milwaukee,202083,Wesley Matthews,F,NaN,27:08,3.0,...,4.0,4.0,8.0,2.0,2.0,0.0,0.0,0.0,8.0,11.0
1,21900895,1610612749,MIL,Milwaukee,203507,Giannis Antetokounmpo,F,NaN,34:55,17.0,...,2.0,18.0,20.0,6.0,1.0,0.0,3.0,2.0,41.0,22.0
2,21900895,1610612749,MIL,Milwaukee,201572,Brook Lopez,C,NaN,26:25,4.0,...,2.0,5.0,7.0,0.0,0.0,3.0,0.0,2.0,16.0,16.0
3,21900895,1610612749,MIL,Milwaukee,1628978,Donte DiVincenzo,G,NaN,27:35,1.0,...,1.0,6.0,7.0,5.0,0.0,1.0,2.0,0.0,2.0,14.0
4,21900895,1610612749,MIL,Milwaukee,202339,Eric Bledsoe,G,NaN,22:17,2.0,...,1.0,0.0,1.0,2.0,1.0,0.0,3.0,2.0,4.0,6.0
5,21900895,1610612749,MIL,Milwaukee,1626192,Pat Connaughton,NaN,NaN,24:52,2.0,...,2.0,3.0,5.0,1.0,0.0,0.0,1.0,2.0,6.0,0.0
6,21900895,1610612749,MIL,Milwaukee,201577,Robin Lopez,NaN,NaN,13:18,1.0,...,1.0,2.0,3.0,0.0,0.0,1.0,2.0,1.0,2.0,-12.0
7,21900895,1610612749,MIL,Milwaukee,1628425,Sterling Brown,NaN,NaN,18:10,1.0,...,0.0,3.0,3.0,0.0,0.0,0.0,1.0,0.0,3.0,-8.0
8,21900895,1610612749,MIL,Milwaukee,101107,Marvin Williams,NaN,NaN,19:37,0.0,...,0.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,0.0,-11.0
9,21900895,1610612749,MIL,Milwaukee,201588,George Hill,NaN,NaN,25:43,4.0,...,2.0,3.0,5.0,2.0,2.0,0.0,3.0,1.0,11.0,2.0


In [22]:
games_details_df.loc[(games_details_df['GAME_ID']==21900895) & (games_details_df['PLAYER_ID'].isin(playerList))]

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
13,21900895,1610612766,CHA,Charlotte,1628970,Miles Bridges,F,NaN,35:15,3.0,...,1.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,7.0,-4.0
14,21900895,1610612766,CHA,Charlotte,1629023,P.J. Washington,F,NaN,31:52,5.0,...,1.0,5.0,6.0,3.0,0.0,2.0,2.0,3.0,12.0,-13.0
15,21900895,1610612766,CHA,Charlotte,202687,Bismack Biyombo,C,NaN,22:07,2.0,...,4.0,5.0,9.0,2.0,0.0,2.0,1.0,3.0,8.0,-15.0
16,21900895,1610612766,CHA,Charlotte,1628984,Devonte' Graham,G,NaN,32:21,7.0,...,0.0,2.0,2.0,3.0,1.0,0.0,0.0,2.0,17.0,-14.0
17,21900895,1610612766,CHA,Charlotte,1626179,Terry Rozier,G,NaN,36:05,6.0,...,2.0,1.0,3.0,4.0,1.0,0.0,2.0,2.0,13.0,-20.0
18,21900895,1610612766,CHA,Charlotte,1628998,Cody Martin,NaN,NaN,29:08,4.0,...,0.0,5.0,5.0,2.0,0.0,1.0,1.0,2.0,11.0,2.0
19,21900895,1610612766,CHA,Charlotte,1629667,Jalen McDaniels,NaN,NaN,9:21,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,2.0,0.0
20,21900895,1610612766,CHA,Charlotte,1626195,Willy Hernangomez,NaN,NaN,20:21,4.0,...,3.0,10.0,13.0,4.0,2.0,0.0,1.0,0.0,10.0,11.0
21,21900895,1610612766,CHA,Charlotte,1628997,Caleb Martin,NaN,NaN,23:30,2.0,...,1.0,3.0,4.0,1.0,1.0,0.0,1.0,3.0,5.0,13.0
22,21900895,1610612766,CHA,Charlotte,201587,Nicolas Batum,NaN,DNP - Coach's Decision,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
games_details_df.loc[(games_details_df['GAME_ID']==21900895) & (games_details_df['TEAM_ID']==1610612749
                                                               ),'TEAM_ABBREVIATION'].unique()[0]

'MIL'

In [24]:
teams_df

,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,1610612738,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,0,1610612740,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
3,0,1610612741,1966,2019,CHI,Bulls,1966,Chicago,United Center,21711.0,Jerry Reinsdorf,Gar Forman,Jim Boylen,Windy City Bulls
4,0,1610612742,1980,2019,DAL,Mavericks,1980,Dallas,American Airlines Center,19200.0,Mark Cuban,Donnie Nelson,Rick Carlisle,Texas Legends
5,0,1610612743,1976,2019,DEN,Nuggets,1976,Denver,Pepsi Center,19099.0,Stan Kroenke,Tim Connelly,Michael Malone,No Affiliate
6,0,1610612745,1967,2019,HOU,Rockets,1967,Houston,Toyota Center,18104.0,Tilman Fertitta,Daryl Morey,Mike D'Antoni,Rio Grande Valley Vipers
7,0,1610612746,1970,2019,LAC,Clippers,1970,Los Angeles,Staples Center,19060.0,Steve Ballmer,Michael Winger,Doc Rivers,Agua Caliente Clippers of Ontario
8,0,1610612747,1948,2019,LAL,Lakers,1948,Los Angeles,Staples Center,19060.0,Jerry Buss Family Trust,Rob Pelinka,Frank Vogel,South Bay Lakers
9,0,1610612748,1988,2019,MIA,Heat,1988,Miami,AmericanAirlines Arena,19600.0,Micky Arison,Pat Riley,Erik Spoelstra,Sioux Falls Skyforce


### 球探部分測試

In [30]:
sasId = teams_df.loc[teams_df['ABBREVIATION']=='SAS','TEAM_ID'].values[0]
sasId

1610612759

In [32]:
games_details_df.loc[games_details_df['TEAM_ID']==sasId]

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
335,21900893,1610612759,SAS,San Antonio,201942,DeMar DeRozan,F,NaN,32:52,6.0,...,0.0,3.0,3.0,9.0,1.0,1.0,3.0,0.0,16.0,20.0
336,21900893,1610612759,SAS,San Antonio,1626168,Trey Lyles,F,NaN,39:48,8.0,...,0.0,9.0,9.0,2.0,4.0,1.0,1.0,3.0,20.0,14.0
337,21900893,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,C,NaN,4:14,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,9.0
338,21900893,1610612759,SAS,San Antonio,1627854,Bryn Forbes,G,NaN,23:43,5.0,...,0.0,1.0,1.0,1.0,0.0,0.0,3.0,2.0,13.0,7.0
339,21900893,1610612759,SAS,San Antonio,1627749,Dejounte Murray,G,NaN,30:23,7.0,...,0.0,2.0,2.0,7.0,3.0,1.0,4.0,1.0,15.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576464,11200024,1610612759,SAS,San Antonio,201986,Nando De Colo,NaN,NaN,28,4.0,...,0.0,2.0,2.0,9.0,2.0,0.0,3.0,2.0,11.0,NaN
576465,11200024,1610612759,SAS,San Antonio,1536,Stephen Jackson,NaN,NaN,15,4.0,...,0.0,2.0,2.0,0.0,1.0,1.0,0.0,1.0,10.0,NaN
576466,11200024,1610612759,SAS,San Antonio,201168,Tiago Splitter,NaN,NaN,13,1.0,...,0.0,6.0,6.0,2.0,0.0,0.0,1.0,0.0,2.0,NaN
576467,11200024,1610612759,SAS,San Antonio,203174,Tyler Wilkerson,NaN,NaN,4,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,NaN


In [33]:
games_df.loc[(games_df['HOME_TEAM_ID']==sasId)|(games_df['VISITOR_TEAM_ID']==sasId)][]

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
13,2020-02-29,21900893,Final,1610612759,1610612753,2019,1610612759,114.0,0.524,0.750,...,28.0,31.0,1610612753,113.0,0.488,0.696,0.405,28.0,41.0,1
35,2020-02-26,21900869,Final,1610612759,1610612742,2019,1610612759,103.0,0.424,0.737,...,22.0,55.0,1610612742,109.0,0.419,0.667,0.378,25.0,43.0,0
57,2020-02-23,21900846,Final,1610612760,1610612759,2019,1610612760,131.0,0.575,0.875,...,28.0,54.0,1610612759,103.0,0.369,0.842,0.289,27.0,43.0,1
73,2020-02-21,21900832,Final,1610612762,1610612759,2019,1610612762,104.0,0.427,0.846,...,25.0,47.0,1610612759,113.0,0.440,0.885,0.500,22.0,48.0,0
98,2020-02-11,21900805,Final,1610612760,1610612759,2019,1610612760,106.0,0.459,0.810,...,19.0,35.0,1610612759,114.0,0.488,0.926,0.280,20.0,46.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23092,2014-10-24,11400113,Final,1610612745,1610612759,2014,1610612745,96.0,0.422,0.647,...,16.0,46.0,1610612759,87.0,0.406,0.793,0.308,24.0,38.0,1
23105,2014-10-22,11400105,Final,1610612759,1610612737,2014,1610612759,107.0,0.532,0.818,...,25.0,38.0,1610612737,117.0,0.519,0.735,0.480,29.0,34.0,0
23120,2014-10-20,11400091,Final,1610612759,1610612758,2014,1610612759,106.0,0.468,0.889,...,24.0,36.0,1610612758,99.0,0.507,0.778,0.125,19.0,39.0,1
23129,2014-10-18,11400078,Final,1610612759,1610612748,2014,1610612759,108.0,0.413,0.833,...,27.0,50.0,1610612748,111.0,0.455,0.723,0.318,20.0,37.0,0


In [86]:
# players_df.loc[players_df['TEAM_ID']==sasId,'PLAYER_NAME']
all_year_list = players_df.loc[players_df['TEAM_ID']==sasId,'SEASON'].unique()
print(all_year_list)
players2019 = list(players_df.loc[(players_df['TEAM_ID']==sasId)&(players_df['SEASON']==all_year_list[0]),'PLAYER_ID'].unique())
players2018 = list(players_df.loc[(players_df['TEAM_ID']==sasId)&(players_df['SEASON']==all_year_list[1]),'PLAYER_ID'].unique())
players2018

[2019 2018 2017 2016 2015 2014 2013 2012 2011 2010 2009]


[201942,
 200746,
 1627751,
 1627854,
 1628401,
 200752,
 201988,
 201158,
 202722,
 201967,
 202700,
 202347,
 1629022,
 1629234,
 1629002,
 2200,
 1627855,
 203910,
 203962,
 1627749,
 1628417]

In [87]:
# 兩年球員差異
def playerDiff(new,old):
    return list(set(old).difference(set(new))) 

In [88]:
print(players2019,'==',len(players2019))
print(players2018,'==',len(players2018))
playerdiff_list = playerDiff(players2019,players2018)
playerdiff_list

[201942, 200752, 1627751, 1628401, 1627854, 1627749, 201988, 1626168, 1629234, 201158, 1629022, 201960, 1629002, 200746, 1629640, 1629683, 1629677, 1629756, 1629664] == 19
[201942, 200746, 1627751, 1627854, 1628401, 200752, 201988, 201158, 202722, 201967, 202700, 202347, 1629022, 1629234, 1629002, 2200, 1627855, 203910, 203962, 1627749, 1628417] == 21


[1628417, 202722, 203910, 202347, 202700, 1627855, 201967, 2200, 203962]

In [102]:
# 2018 存在 2019 不存在 =>離開team
players_df.loc[(players_df['PLAYER_ID'].isin(playerdiff_list)) & (players_df['SEASON']==2019)& (players_df['TEAM_ID']==sasId)]

,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON


In [103]:
players_df.loc[(players_df['PLAYER_ID'].isin(playerdiff_list)) & (players_df['SEASON']==2018) & (players_df['TEAM_ID']==sasId)]

,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON
747,Davis Bertans,1610612759,202722,2018
748,Dante Cunningham,1610612759,201967,2018
749,Donatas Motiejunas,1610612759,202700,2018
750,Quincy Pondexter,1610612759,202347,2018
1108,Pau Gasol,1610612759,2200,2018
1287,Okaro White,1610612759,1627855,2018
1288,Nick Johnson,1610612759,203910,2018
1324,Josh Huestis,1610612759,203962,2018
1357,Jaron Blossomgame,1610612759,1628417,2018


In [114]:
games_df['GAME_DATE_EST'] = pd.to_datetime(games_df['GAME_DATE_EST'])
games_df.loc[(games_df['SEASON']==2018)]


,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
16196,2019-06-13,41800406,Final,1610612744,1610612761,2018,1610612744,110.0,0.488,0.700,...,28.0,42.0,1610612761,114.0,0.476,0.793,0.394,25.0,39.0,0
16197,2019-06-10,41800405,Final,1610612761,1610612744,2018,1610612761,105.0,0.447,0.778,...,19.0,43.0,1610612744,106.0,0.463,0.714,0.476,27.0,37.0,0
16198,2019-06-07,41800404,Final,1610612744,1610612761,2018,1610612744,92.0,0.449,0.667,...,26.0,42.0,1610612761,105.0,0.419,0.958,0.313,22.0,39.0,0
16199,2019-06-05,41800403,Final,1610612744,1610612761,2018,1610612744,109.0,0.396,0.833,...,25.0,41.0,1610612761,123.0,0.524,0.952,0.447,30.0,40.0,0
16200,2019-06-02,41800402,Final,1610612761,1610612744,2018,1610612761,104.0,0.372,0.885,...,17.0,49.0,1610612744,109.0,0.463,0.870,0.382,34.0,42.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17569,2018-09-30,11800009,Final,1610612741,1610612740,2018,1610612741,128.0,0.495,0.963,...,23.0,52.0,1610612740,116.0,0.454,0.789,0.333,24.0,45.0,1
17570,2018-09-30,11800010,Final,1610612747,1610612743,2018,1610612747,107.0,0.460,0.731,...,26.0,37.0,1610612743,124.0,0.460,0.886,0.351,25.0,50.0,0
17571,2018-09-29,11800003,Final,1610612761,1610612757,2018,1610612761,122.0,0.415,0.824,...,15.0,45.0,1610612757,104.0,0.420,0.720,0.387,25.0,46.0,1
17572,2018-09-29,11800005,Final,1610612744,1610612750,2018,1610612744,110.0,0.473,0.769,...,33.0,48.0,1610612750,114.0,0.426,0.733,0.400,15.0,47.0,0


### <font style="color:red;">待做 針對 sas 切 game_df['SEASON'] ==2019~2015  的五年分 df</font>
### <font style="color:red;">待做 針對 SAS 球員教義後的數據變化  => 球員數據 和 game 數據  和 range 排名。</font>